Se instala la librería backtesting.py

In [1]:
!pip install backtesting --quiet


Se vuelven a declarar las funciones de métricas personalizadas para no tener problemas a la hora de cargar el modelo.

In [2]:
from backtesting import Strategy
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

def f1_metric(y_true, y_pred):

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))  # mistake: y_pred of 0.3 is also considered 1
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)

    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

get_custom_objects().update({"f1_metric": f1_metric})


/opt/conda/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Se carga un conjunto de datos desde un archivo HDF5 usando Pandas. La ruta del archivo es /kaggle/input/data-backtesting/data_backtesting.h5, y el conjunto de datos específico que se carga es 'data_backtesting'. Finalmente, muestra el contenido del conjunto de datos cargado.

In [3]:
with pd.HDFStore('/kaggle/input/data-backtesting/data_backtesting.h5', mode='r') as store:
    data = store.get('data_backtesting')
data

,Open,High,Low,Close,Volume,06_RSI,07_RSI,08_RSI,09_RSI,10_RSI,...,11_MFI,12_MFI,13_MFI,14_MFI,15_MFI,16_MFI,17_MFI,18_MFI,19_MFI,20_MFI
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,2476.959961,2519.489990,2467.469971,2510.030029,3733160000,51.366141,48.674962,46.753118,45.352718,44.314822,...,31.240747,28.761515,26.656690,31.734762,36.059801,33.826957,31.796038,29.611699,27.905279,31.667857
2019-01-03,2491.919922,2493.139893,2443.959961,2447.889893,3858830000,36.564095,36.604779,36.544063,36.482625,36.450188,...,31.870245,29.001798,26.852972,25.009267,29.901454,34.101451,32.097808,30.263584,28.278152,26.717917
2019-01-04,2474.330078,2538.070068,2474.330078,2531.939941,4234140000,56.779473,54.434865,52.557933,51.050376,49.831139,...,40.995440,37.396872,34.281296,31.917148,29.867716,34.164695,37.882907,35.777709,33.837992,31.724740
2019-01-07,2535.610107,2566.159912,2524.560059,2549.689941,4133120000,60.009081,57.387700,55.281651,53.579790,52.193144,...,50.845905,45.763316,42.044449,38.781538,36.278361,34.089106,37.898416,41.217475,39.043445,37.028758
2019-01-08,2568.110107,2579.820068,2547.560059,2574.409912,4120060000,64.448703,61.447083,59.025779,57.056512,55.439553,...,61.611502,54.913390,49.836445,46.067725,42.722749,40.131948,37.848270,41.246652,44.226048,42.006730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,3046770000,68.692349,69.567254,70.239093,70.725499,71.045350,...,74.906871,76.655095,71.165077,72.950572,68.231518,70.180961,65.287922,67.162991,68.540014,65.917442
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,2513910000,72.543015,72.824566,73.066377,73.227672,73.294261,...,74.353623,76.132782,77.719762,72.389940,74.031284,69.421741,71.232009,66.428937,68.185882,69.480141
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,2748450000,73.853049,73.929679,74.023762,74.073948,74.054362,...,73.838129,75.747326,77.344389,78.779178,73.616755,75.119447,70.626420,72.300766,67.594957,69.234955


Se crea dos subconjuntos de datos a partir del conjunto de datos 'data':

1) ohlc: Contiene las columnas 'Open', 'High', 'Low', 'Close' y 'Volume', que son típicamente utilizadas en análisis financieros para representar la información de precios y volumen de un activo.

2) img: Extrae todas las columnas a partir de la sexta columna en adelante, asumiendo que estas contienen datos distintos de los precios OHLC y volumen, posiblemente características adicionales o datos procesados.

In [4]:
ohlc = data[['Open', 'High', 'Low', 'Close', 'Volume']]
img = data.iloc[:, 5:]

In [5]:
ohlc

,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02,2476.959961,2519.489990,2467.469971,2510.030029,3733160000
2019-01-03,2491.919922,2493.139893,2443.959961,2447.889893,3858830000
2019-01-04,2474.330078,2538.070068,2474.330078,2531.939941,4234140000
2019-01-07,2535.610107,2566.159912,2524.560059,2549.689941,4133120000
2019-01-08,2568.110107,2579.820068,2547.560059,2574.409912,4120060000
...,...,...,...,...,...
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,3046770000
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,2513910000
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,2748450000


In [6]:
img

,06_RSI,07_RSI,08_RSI,09_RSI,10_RSI,11_RSI,12_RSI,13_RSI,14_RSI,15_RSI,...,11_MFI,12_MFI,13_MFI,14_MFI,15_MFI,16_MFI,17_MFI,18_MFI,19_MFI,20_MFI
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,51.366141,48.674962,46.753118,45.352718,44.314822,43.534272,42.939850,42.482398,42.127432,41.850385,...,31.240747,28.761515,26.656690,31.734762,36.059801,33.826957,31.796038,29.611699,27.905279,31.667857
2019-01-03,36.564095,36.604779,36.544063,36.482625,36.450188,36.450877,36.480414,36.532622,36.601730,36.683040,...,31.870245,29.001798,26.852972,25.009267,29.901454,34.101451,32.097808,30.263584,28.278152,26.717917
2019-01-04,56.779473,54.434865,52.557933,51.050376,49.831139,48.836753,48.018574,47.339587,46.771618,46.293119,...,40.995440,37.396872,34.281296,31.917148,29.867716,34.164695,37.882907,35.777709,33.837992,31.724740
2019-01-07,60.009081,57.387700,55.281651,53.579790,52.193144,51.052894,50.106531,49.314113,48.645167,48.076303,...,50.845905,45.763316,42.044449,38.781538,36.278361,34.089106,37.898416,41.217475,39.043445,37.028758
2019-01-08,64.448703,61.447083,59.025779,57.056512,55.439553,54.098728,52.976184,52.027919,51.220276,50.527309,...,61.611502,54.913390,49.836445,46.067725,42.722749,40.131948,37.848270,41.246652,44.226048,42.006730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,68.692349,69.567254,70.239093,70.725499,71.045350,71.218880,71.267119,71.210806,71.069402,70.860436,...,74.906871,76.655095,71.165077,72.950572,68.231518,70.180961,65.287922,67.162991,68.540014,65.917442
2023-12-26,72.543015,72.824566,73.066377,73.227672,73.294261,73.265980,73.150397,72.958932,72.704374,72.399368,...,74.353623,76.132782,77.719762,72.389940,74.031284,69.421741,71.232009,66.428937,68.185882,69.480141
2023-12-27,73.853049,73.929679,74.023762,74.073948,74.054362,73.957651,73.786667,73.549586,73.256889,72.919544,...,73.838129,75.747326,77.344389,78.779178,73.616755,75.119447,70.626420,72.300766,67.594957,69.234955


Se selecciona un subconjunto específico de columnas, identificadas por los índices en feat_idx(calculado en el notebook feature_engineering), del dataframe img. Luego, muestra el contenido del nuevo dataframe imgs que contiene solo las columnas seleccionadas. Este paso es útil para enfocarse en características específicas dentro de un conjunto de datos más amplio.

In [7]:
feat_idx = [0, 1, 3, 4, 5, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 24, 25, 26, 27, 56, 59, 62, 65, 68, 74, 76, 77, 79, 80, 83, 84, 86, 87, 89, 90, 91, 92, 93, 96, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 137, 138, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 163, 164, 166, 168, 170, 172, 173, 174, 175, 178, 179, 180, 181, 182, 184, 185, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 201, 202, 203, 204, 205, 206, 208, 209, 212, 213, 215, 216, 217, 218, 220, 221, 224, 225, 227, 228, 229, 230, 231, 232, 233, 235, 236, 240, 241, 242, 244, 246, 247, 248, 249, 250, 251, 253, 254, 255, 256, 257, 258, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 276, 277, 285, 286, 287, 288, 289, 290, 291, 293, 296, 297, 298, 301, 302, 305, 306, 307, 308, 309, 310, 311, 312, 313, 315, 316, 318, 319, 320, 322, 323, 324, 325, 326, 327, 328, 329, 331, 334, 336, 337, 338, 340, 341, 342, 344]
imgs = img.iloc[:, feat_idx]
imgs



,06_RSI,07_RSI,09_RSI,10_RSI,11_RSI,13_RSI,14_RSI,16_RSI,17_RSI,18_RSI,...,20_ULTOSC,07_MFI,10_MFI,12_MFI,13_MFI,14_MFI,16_MFI,17_MFI,18_MFI,20_MFI
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,51.366141,48.674962,45.352718,44.314822,43.534272,42.482398,42.127432,41.633510,41.463798,41.331586,...,47.162300,52.892854,34.594790,28.761515,26.656690,31.734762,33.826957,31.796038,29.611699,31.667857
2019-01-03,36.564095,36.604779,36.482625,36.450188,36.450877,36.532622,36.601730,36.772964,36.868857,36.968813,...,45.124559,60.569080,35.236335,29.001798,26.852972,25.009267,34.101451,32.097808,30.263584,26.717917
2019-01-04,56.779473,54.434865,51.050376,49.831139,48.836753,47.339587,46.771618,45.887492,45.541841,45.246049,...,47.335919,72.277291,45.997588,37.396872,34.281296,31.917148,34.164695,37.882907,35.777709,31.724740
2019-01-07,60.009081,57.387700,53.579790,52.193144,51.052894,49.314113,48.645167,47.589413,47.170357,46.807974,...,49.461365,72.363011,57.682624,45.763316,42.044449,38.781538,34.089106,37.898416,41.217475,37.028758
2019-01-08,64.448703,61.447083,57.056512,55.439553,54.098728,52.027919,51.220276,49.928832,49.408980,48.955162,...,48.267468,72.524736,73.717928,54.913390,49.836445,46.067725,40.131948,37.848270,41.246652,42.006730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,68.692349,69.567254,70.725499,71.045350,71.218880,71.210806,71.069402,70.599202,70.298722,69.969861,...,63.452253,62.794410,72.932692,76.655095,71.165077,72.950572,70.180961,65.287922,67.162991,65.917442
2023-12-26,72.543015,72.824566,73.227672,73.294261,73.265980,72.958932,72.704374,72.055577,71.683286,71.291277,...,63.138803,58.058751,72.208992,76.132782,77.719762,72.389940,69.421741,71.232009,66.428937,69.480141
2023-12-27,73.853049,73.929679,74.073948,74.054362,73.957651,73.549586,73.256889,72.547971,72.151521,71.738269,...,64.231503,82.606476,71.602260,75.747326,77.344389,78.779178,75.119447,70.626420,72.300766,69.234955


Se realiza varios pasos para preparar datos en formato de imagen:

1) Define la función reshape_as_image para convertir un array 1D en un array 3D (imagen).

2) Normaliza los datos en imgs usando MinMaxScaler para que los valores estén en el rango de 0 a 1, lo cual es una práctica común en el procesamiento de imágenes.

3) Utiliza reshape_as_image para redimensionar los datos normalizados a un formato de imagen de 15x15.

4) Agrega una dimensión adicional para simular canales de color (RGB) aunque las imágenes son esencialmente en escala de grises.

5) Muestra la forma final del array de imágenes, indicando el número de imágenes y sus dimensiones.








In [8]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def reshape_as_image(x, img_width, img_height):
    x_temp = np.zeros((len(x), img_height, img_width))
    for i in range(x.shape[0]):
        x_temp[i] = np.reshape(x[i], (img_height, img_width))

    return x_temp

# Seleccionar las columnas necesarias para formar la imagen
num_features = 225  # 15x15
img_width = img_height = 15

# Normalizar imágenes
mm_scaler = MinMaxScaler(feature_range=(0, 1))
imgs = mm_scaler.fit_transform(imgs)

# Redimensionar como imágenes
imagenes = reshape_as_image(imgs, img_width, img_height)

# Añadir una dimensión para canales (3)
imagenes = np.stack((imagenes,) * 3, axis=-1)

# Mostrar la forma de las imágenes
imagenes.shape


(1258, 15, 15, 3)

Se carga un modelo previamente guardado desde una ubicación específica y luego utiliza este modelo para realizar predicciones en el conjunto de imágenes procesadas y preparadas en los pasos anteriores.

In [9]:
loaded_model = load_model('/kaggle/input/ensayo-1-sample-weights/mi_modelo.h5')
pred = loaded_model.predict(imagenes)

40/40 [==============================] - 0s 4ms/step


Se define y aplica una función apply_threshold para ajustar las predicciones del modelo basándose en un umbral específico, en este caso 0.6. La función ajusta las predicciones asignando la clase 0 si ninguna clase supera el umbral, o la clase con la mayor probabilidad si alguna lo supera. Finalmente, convierte estas predicciones ajustadas en índices de clases y las muestra.

In [10]:
def apply_threshold(predictions, threshold):
    adjusted_predictions = []
    for pred in predictions:
        if np.max(pred) < threshold:
            # Si ninguna clase supera el umbral, asignamos a la clase 0
            adjusted_pred = [1, 0, 0]
        else:
            # Si alguna clase supera el umbral, seleccionamos la de mayor probabilidad
            adjusted_pred = [0, 0, 0]  # Inicialmente, todas las clases se establecen en 0
            adjusted_pred[np.argmax(pred)] = 1  # Solo la clase con la mayor probabilidad se establece en 1
        adjusted_predictions.append(adjusted_pred)
    return np.array(adjusted_predictions)

pred_umb_060 = apply_threshold(pred, 0.6)
pred_umb_060 = np.argmax(pred_umb_060, axis=1)
pred_umb_060

array([0, 0, 0, ..., 2, 0, 0])

Se aplica la misma función apply_threshold con diferentes umbrales (0.65, 0.7, 0.75, 0.8) a las predicciones del modelo. Para cada umbral, las predicciones son ajustadas y luego convertidas en índices de clases. Esto permite evaluar el impacto de diferentes umbrales en la clasificación final del modelo.

In [11]:
pred_umb_065 = apply_threshold(pred, 0.65)
pred_umb_065 = np.argmax(pred_umb_065, axis=1)

pred_umb_070 = apply_threshold(pred, 0.7)
pred_umb_070 = np.argmax(pred_umb_070, axis=1)

pred_umb_075 = apply_threshold(pred, 0.75)
pred_umb_075 = np.argmax(pred_umb_075, axis=1)

pred_umb_080 = apply_threshold(pred, 0.8)
pred_umb_080 = np.argmax(pred_umb_080, axis=1)

Esta celda añade las predicciones ajustadas con diferentes umbrales (0.60, 0.65, 0.70, 0.75, 0.80) como nuevas columnas al dataframe ohlc. Cada columna representa las clases predichas para cada umbral aplicado, proporcionando una visión detallada de cómo varían las predicciones del modelo al cambiar el umbral.

In [12]:
ohlc['pred_umb_060'] = pred_umb_060
ohlc['pred_umb_065'] = pred_umb_065
ohlc['pred_umb_070'] = pred_umb_070
ohlc['pred_umb_075'] = pred_umb_075
ohlc['pred_umb_080'] = pred_umb_080

/tmp/ipykernel_19/520438220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ohlc['pred_umb_060'] = pred_umb_060
/tmp/ipykernel_19/520438220.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ohlc['pred_umb_065'] = pred_umb_065
/tmp/ipykernel_19/520438220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [13]:
ohlc

,Open,High,Low,Close,Volume,pred_umb_060,pred_umb_065,pred_umb_070,pred_umb_075,pred_umb_080
Date,,,,,,,,,,
2019-01-02,2476.959961,2519.489990,2467.469971,2510.030029,3733160000,0,0,0,0,0
2019-01-03,2491.919922,2493.139893,2443.959961,2447.889893,3858830000,0,0,0,0,0
2019-01-04,2474.330078,2538.070068,2474.330078,2531.939941,4234140000,0,0,0,0,0
2019-01-07,2535.610107,2566.159912,2524.560059,2549.689941,4133120000,0,0,0,0,0
2019-01-08,2568.110107,2579.820068,2547.560059,2574.409912,4120060000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,3046770000,0,0,0,0,0
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,2513910000,0,0,0,0,0
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,2748450000,2,0,0,0,0


Se implementa un proceso de optimización para una estrategia de trading basada en predicciones de modelos de aprendizaje automático:

1) Definición de la Clase de Estrategia (create_strategy_class): Crea una clase de estrategia personalizada para el backtesting. La estrategia utiliza umbrales para tomar decisiones de compra o venta y tiene parámetros de take profit (TP) y stop loss (SL).

2) Función de Optimización (optimize_strategy): Realiza un proceso de optimización iterando sobre rangos de valores para TP, SL y umbrales de predicción. Utiliza el rendimiento ajustado por drawdown como métrica para encontrar la mejor configuración.

3) Ejecución del Proceso de Optimización: Define rangos de valores para TP y SL, así como los nombres de las columnas de umbral. Luego ejecuta la función optimize_strategy con estos valores y el dataframe ohlc.

4) Resultados: Imprime la mejor configuración encontrada, basada en el máximo rendimiento ajustado por drawdown.

Este enfoque permite encontrar la configuración óptima para maximizar los retornos ajustados por el riesgo en una estrategia de trading algorítmico.








In [14]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np
from tqdm import tqdm

def create_strategy_class(tp_pct_buy, tp_pct_sell, sl_pct_buy, sl_pct_sell, threshold):
    class MLmodel(Strategy):
        def init(self):
            # Inicialización de parámetros
            self.tp_pct_buy = tp_pct_buy
            self.tp_pct_sell = tp_pct_sell
            self.sl_pct_buy = sl_pct_buy
            self.sl_pct_sell = sl_pct_sell
            self.threshold = threshold

        def next(self):
            # Precio actual
            precio_actual = self.data.Close[-1]

            # Calcular precios de TP y SL
            precio_tp_buy = precio_actual * (1 + self.tp_pct_buy)
            precio_sl_buy = precio_actual * (1 - self.sl_pct_buy)
            precio_tp_sell = precio_actual * (1 - self.tp_pct_sell)
            precio_sl_sell = precio_actual * (1 + self.sl_pct_sell)

            # Lógica de trading basada en el umbral
            if getattr(self.data, self.threshold)[-1] == 1:
                if not self.position.is_long:
                    self.position.close()
                    self.buy(sl=precio_sl_buy, tp=precio_tp_buy)

            elif getattr(self.data, self.threshold)[-1] == 2:
                if not self.position.is_short:
                    self.position.close()
                    self.sell(sl=precio_sl_sell, tp=precio_tp_sell)

    return MLmodel

# Función para realizar la optimización
def optimize_strategy(data, tp_buy_values, tp_sell_values, sl_buy_values, sl_sell_values, thresholds):
    best_result = {'return_over_drawdown': -np.inf, 'tp_buy': 0, 'tp_sell': 0, 'sl_buy': 0, 'sl_sell': 0, 'threshold': ''}
    
    total_iterations = len(tp_buy_values) * len(tp_sell_values) * len(sl_buy_values) * len(sl_sell_values) * len(thresholds)
    
    # Inicializar la barra de progreso de tqdm
    pbar = tqdm(total=total_iterations, desc="Optimizing", leave=True)
    
    for tp_buy in tp_buy_values:
        for tp_sell in tp_sell_values:
            for sl_buy in sl_buy_values:
                for sl_sell in sl_sell_values:
                    for threshold in thresholds:
                        StrategyClass = create_strategy_class(tp_buy, tp_sell, sl_buy, sl_sell, threshold)
                        bt = Backtest(data, StrategyClass, cash=10000, commission=.0005)
                        stats = bt.run()
                        return_over_drawdown = stats['Return [%]'] / abs(stats['Max. Drawdown [%]'])

                        if return_over_drawdown > best_result['return_over_drawdown']:
                            best_result = {'return_over_drawdown': return_over_drawdown, 
                                           'tp_buy': tp_buy, 'tp_sell': tp_sell, 
                                           'sl_buy': sl_buy, 'sl_sell': sl_sell, 
                                           'threshold': threshold}
                            
                        # Actualizar la barra de progreso
                        pbar.update(1)
                        
    pbar.close()  # Cerrar la barra de progreso
    return best_result

# Ejecución del optimizador
tp_buy_values = np.arange(0.01, 0.11, 0.01)  # TP de compra desde 1% hasta 10%
tp_sell_values = np.arange(0.01, 0.11, 0.01)  # TP de venta desde 1% hasta 10%
sl_buy_values = np.arange(0.01, 0.06, 0.01)   # SL de compra desde 1% hasta 5%
sl_sell_values = np.arange(0.01, 0.06, 0.01)  # SL de venta desde 1% hasta 5%
thresholds = ['pred_umb_060', 'pred_umb_065', 'pred_umb_070', 'pred_umb_075', 'pred_umb_080']


best_configuration = optimize_strategy(ohlc, tp_buy_values, tp_sell_values, sl_buy_values, sl_sell_values, thresholds)
print("Mejor configuración:", best_configuration)



Optimizing: 100%|██████████| 12500/12500 [22:25<00:00,  9.29it/s]

Mejor configuración: {'return_over_drawdown': 9.32671501773394, 'tp_buy': 0.060000000000000005, 'tp_sell': 0.04, 'sl_buy': 0.01, 'sl_sell': 0.02, 'threshold': 'pred_umb_075'}


Se aplica la mejor configuración encontrada en el proceso de optimización a la estrategia de trading:

1) Creación de la Clase de Estrategia: Utiliza la función create_strategy_class para crear una clase de estrategia (StrategyClass) con los parámetros de TP, SL y umbral obtenidos de la mejor configuración.

2) Ejecución del Backtest: Utiliza la biblioteca Backtest para probar la estrategia creada con el dataframe ohlc, un capital inicial de 10,000 y una comisión específica.

3) Resultados del Backtest: Ejecuta el backtest con la estrategia configurada y almacena las estadísticas de rendimiento en stats.

In [15]:
StrategyClass = create_strategy_class(tp_pct_buy=best_configuration['tp_buy'], 
                                      tp_pct_sell = best_configuration['tp_sell'], 
                                      sl_pct_buy = best_configuration['sl_buy'], 
                                      sl_pct_sell = best_configuration['sl_sell'], 
                                      threshold = best_configuration['threshold'])

bt = Backtest(ohlc, StrategyClass, cash=10000, commission=.0005)
stats = bt.run()

In [16]:
stats

Start                     2019-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   1822 days 00:00:00
Exposure Time [%]                   62.559618
Equity Final [$]                 17541.752203
Equity Peak [$]                  17684.481852
Return [%]                          75.417522
Buy & Hold Return [%]               90.030797
Return (Ann.) [%]                   11.916006
Volatility (Ann.) [%]               11.268191
Sharpe Ratio                         1.057491
Sortino Ratio                         1.82337
Calmar Ratio                         1.473626
Max. Drawdown [%]                   -8.086183
Avg. Drawdown [%]                   -1.811194
Max. Drawdown Duration      231 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                  145
Win Rate [%]                        35.862069
Best Trade [%]                       6.301045
Worst Trade [%]                     -3.403059
Avg. Trade [%]                    

Se extraen los trades realizados.

In [17]:
trades = stats['_trades']

In [18]:
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,3,44,44,2767.913294,2743.735452,-72.533528,-0.008735,2019-03-07,2019-03-07,0 days
1,3,45,49,2732.155434,2810.379883,234.673346,0.028631,2019-03-08,2019-03-14,6 days
2,-3,49,56,2808.974693,2796.010010,38.894049,0.004615,2019-03-14,2019-03-25,11 days
3,3,56,62,2797.408015,2868.239990,212.495926,0.025321,2019-03-25,2019-04-02,8 days
4,-3,62,76,2866.805870,2924.533740,-173.183610,-0.020137,2019-04-02,2019-04-23,21 days
...,...,...,...,...,...,...,...,...,...,...
140,3,1197,1212,4235.946993,4187.155693,-146.373899,-0.011518,2023-10-04,2023-10-25,21 days
141,3,1213,1213,4178.078229,4144.902319,-99.527730,-0.007940,2023-10-26,2023-10-26,0 days
142,3,1214,1219,4155.006641,4334.229980,537.670019,0.043134,2023-10-27,2023-11-03,7 days
143,-3,1219,1224,4332.062865,4404.135381,-216.217546,-0.016637,2023-11-03,2023-11-10,7 days


Se plotean tanto los trades realizados como el porcentaje de beneficios generados a lo largo del tiempo.

In [19]:
bt.plot(plot_volume=False, plot_pl=False, plot_return = True, plot_equity = False)

/opt/conda/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/conda/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/conda/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/conda/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1206', ...)